# 10장. apply 메소드 활용
apply 메소드는 사용자가 작성한 함수를 한 번에 데이터프레임의 각 행과 열에 적용하여 실행할 수 있도록 해주는 함수이다. 

물론 for 문을 사용하여 적용할 수 있지만, apply가 훨씬 빠르기 때문에 사용법을 익혀두는 것이 좋다.

## 10.1 간단한 함수 생성
### 10.1.1 제곱 함수와 n 제곱 함수 만들기(202쪽)

In [1]:
def my_sq(x):
    return x ** 2

In [2]:
def my_exp(x, n):
    return x ** n

In [3]:
print(my_sq(4))

16


In [4]:
print(my_exp(2, 4))

16


<font color='red'>[Quiz] </font> 위 함수를 lambda 함수로 작성해 보세요.

In [5]:
my_sq = lambda x: x ** 2
my_exp = lambda x, n: x ** n 

## 10.2 apply 메소드 사용하기
### 10.2.1 시리즈와 데이터프레임에 apply 메서드 사용(203쪽)

#### 1) 시리즈와 apply 메서드

In [6]:
import pandas as pd

df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]}) 
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [7]:
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
sq = df['a'].apply(my_sq) 
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [9]:
df['a'].map(my_sq) 

0    100
1    400
2    900
Name: a, dtype: int64

In [10]:
ex = df['a'].apply(my_exp, n=2)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64


In [11]:
ex = df['a'].apply(my_exp, n=3) 
print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


In [20]:
import numpy as np
df.a.mean()
df.a.apply('mean')
df.a.apply(['mean', 'sum'])

mean    20.0
sum     60.0
Name: a, dtype: float64

#### 2) 데이터 프레임과 apply 메서드
이때, axis=0은 열방향을 지정하며, axis=1은 행방향을 지정한다.

In [21]:
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]}) 
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [22]:
df.apply(my_sq)

,a,b
0,100,400
1,400,900
2,900,1600


In [23]:
def print_me(x): 
    print(x)

In [24]:
print(df.apply(print_me, axis=0))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [25]:
print(df['a'])

0    10
1    20
2    30
Name: a, dtype: int64


In [26]:
print(df['b'])

0    20
1    30
2    40
Name: b, dtype: int64


In [27]:
print(df.apply(print_me, axis=1))

a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64
0    None
1    None
2    None
dtype: object


In [28]:
def avg_3(x, y, z):
    return (x + y + z) / 3

그런데 avg_3 함수를 apply 메서드에 전달하면 
- avg_3 함수는 3개의 인잣값을 필요로 하는 함수인데 "1개의 인잣값만 입력받았다"는 오류 메시지가 출력됩니다. 
- 즉, avg_3 함수에 열 단위 데이터(df['a'] 또는 df['b'])가 전달되었고 
- 이 값을 avg_3 함수에서 1개의 인자로 인식한 것입니다. 

따라서 avg_3 함수가 열 단위로 데이터를 처리할 수 있도록 수정해야 합니다.

In [29]:
try:
    df.apply(avg_3)
except TypeError as e:
    print(e)

("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')


수정한 함수를 적용하면:

In [30]:
def avg_3_apply(col):
    x = col[0] 
    y = col[1] 
    z = col[2] 
    return (x + y + z) / 3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [31]:
def avg_3_apply(col):
    sum = 0
    for item in col:
         sum += item
    return sum / df.shape[0]

<font color='red'>[Quiz] </font> 위 함수를 numpy 함수로 일반화하면:

In [42]:
def avg(col):
    sum = col.values.sum()
    N = col.shape[0]
    return sum / N

In [44]:
df.apply(avg, axis=1)

0    15.0
1    25.0
2    35.0
dtype: float64

행방향으로 수행하는 함수를 다음과 같이 작성할 수 있다.

In [31]:
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum += item
    return sum / df.shape[1]

In [32]:
print(df.apply(avg_2_apply, axis = 1))

0    15.0
1    25.0
2    35.0
dtype: float64


## 10.3 apply 고급 사용법
### 10.3.1 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용(208쪽)

#### 1) 데이터프레임의 누락값 처리하기 ― 열 방향

In [45]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [46]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [48]:
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [49]:
def count_missing(vec): return pd.isnull(vec).sum()

In [50]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [51]:
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

In [52]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [53]:
def prop_complete(vec):
    return 1 - prop_missing(vec)

#### 2)  데이터프레임의 누락값을 처리하기 ― 행 방뱡

In [54]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [55]:
print(pmis_row.head())

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64


In [56]:
print(pcom_row.head())

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [57]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)

print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [60]:
print(titanic.loc[titanic.num_missing > 1, :].sample(10))
# titanic[titanic.num_missing > 1].sample(10)

     survived  pclass     sex  age  sibsp  parch     fare embarked  class  \
846         0       3    male  NaN      8      2  69.5500        S  Third   
495         0       3    male  NaN      0      0  14.4583        C  Third   
692         1       3    male  NaN      0      0  56.4958        S  Third   
451         0       3    male  NaN      1      0  19.9667        S  Third   
359         1       3  female  NaN      0      0   7.8792        Q  Third   
502         0       3  female  NaN      0      0   7.6292        Q  Third   
240         0       3  female  NaN      1      0  14.4542        C  Third   
760         0       3    male  NaN      0      0  14.5000        S  Third   
19          1       3  female  NaN      0      0   7.2250        C  Third   
522         0       3    male  NaN      0      0   7.2250        C  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
846    man        True  NaN  Southampton    no  False            2  
495    man    